# Lib

In [1]:
import torch 
import torch.nn as nn 
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np 
import os 
import glob
import seaborn as sns 
import pandas as pd


In [2]:
loaded_dict = torch.load("model_archive/Teacher_wReward.pt")

In [8]:
model_state_dict = loaded_dict["model_state_dict"]
for k,v in model_state_dict.items():
    model_state_dict[k] = v.cpu()
for k,v in model_state_dict.items():
    print(k)
    print(v)


std
tensor([0.2119, 0.4276, 0.3166, 0.2169, 0.4184, 0.3089, 0.2140, 0.3638, 0.3230,
        0.2145, 0.3578, 0.3238], device='cuda:1')
student_std
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], device='cuda:1')
teacher_adaptation_module.0.weight
tensor([[ 0.0571, -0.0828,  0.0389,  ...,  0.0731, -0.0907, -0.0630],
        [-0.0144, -0.1337,  0.0726,  ..., -0.0442,  0.0269,  0.0711],
        [-0.0165, -0.0570, -0.0246,  ..., -0.0269,  0.0150,  0.0206],
        ...,
        [-0.0073,  0.0986, -0.0531,  ...,  0.0126, -0.0470, -0.0256],
        [-0.1129, -0.1360, -0.1381,  ...,  0.0008, -0.0236,  0.0243],
        [-0.0210,  0.1404, -0.0999,  ...,  0.0286,  0.0093,  0.0475]],
       device='cuda:1')
teacher_adaptation_module.0.bias
tensor([ 0.1928,  0.0221,  0.0526, -0.1079,  0.1352,  0.0096,  0.0103, -0.0011,
         0.0109, -0.0045,  0.0599, -0.0402,  0.0634, -0.0484,  0.0094, -0.0716,
        -0.1205,  0.0507, -0.0905,  0.1256,  0.0864, -0.0984,  0.1774, -0.0015,
        -0.038

# Func

In [3]:
def check_data(data):
    for k,v in data.items():
        if isinstance(v,np.ndarray):
            print(k,v.shape)
        else:
            try:
                print(k, 'Length: ', len(v))
            except:
                print(k, v)

def load_data(path):
    data = np.load(path,allow_pickle=True).item()
    return data 

# Eval

In [2]:
def get_eval(data):
    res = {}
    name = data['eval_name']
    n_agent = len(data['first_done']) 
    res['eval_name'] = name
    list_act_fluctation, list_tracking_error, list_path_length, list_cot = [], [], [], []
    list_fall_rate = []
    for i in range(n_agent):
        first_done = data['first_done'][i]
        act_fluctation = data['action_fluctation'][i]
        tracking_error = data['tracking_error'][i]
        cot = data['cost_of_transport'][i]
        list_path_length.append(first_done)
        list_act_fluctation.append(act_fluctation[0: first_done])
        list_tracking_error.append(tracking_error[0: first_done])
        list_cot.append(cot[0: first_done])
        list_fall_rate.append(data['Fall'][i])
    
    list_act_fluctation = np.concatenate(list_act_fluctation)
    list_tracking_error = np.concatenate(list_tracking_error)
    list_path_length = np.array(list_path_length)
    #! cot 需要除 mg
    list_cot = np.concatenate(list_cot)/ (11 * 9.8)
    list_fall_rate = np.array(list_fall_rate)

    res['act_fluctation_mean'] = np.mean(list_act_fluctation)
    res['act_fluctation_std'] = np.std(list_act_fluctation)
    res['tracking_error_mean'] = np.mean(list_tracking_error)
    res['tracking_error_std'] = np.std(list_tracking_error)
    res['path_length_mean'] = np.mean(list_path_length)
    res['path_length_std'] = np.std(list_path_length)
    res['cot_mean'] = np.mean(list_cot)
    res['cot_std'] = np.std(list_cot)
    res['fall_rate'] = np.mean(list_fall_rate)
    return res 

def eval_exp(log_path, name = 'noise_level-1'):
    for path in glob.glob(log_path + '/*.npy'):
        if name in path:
            data = np.load(path,allow_pickle=True).item()
            res = get_eval(data)
            # return res

def save_eval_data(log_path, name ):
    list_data = []
    for path in glob.glob(log_path + '/*.npy'):
        if name in path:
            data = np.load(path,allow_pickle=True).item()
            res = get_eval(data)
            list_data.append(res)
    list_res = []
    for d in list_data:
        r = {}
        for key in list(d.keys()):
            if '_mean' in key:
                raw_key = key.replace('_mean', '')
                std_key = key.replace('_mean', '_std')
                mean ,std = np.round(d[key],3), np.round(d[std_key],3)
                r[raw_key] = f"{mean:.3f}({std:.3f})"
                del d[std_key]
            elif '_std' not in key:
                r[key] = f"{d[key]}"
        list_res.append(r)
    # create dataframe
    df = pd.DataFrame(list_res).round(4)
    # write to csv
    df.to_csv(os.path.join(log_path,f'{name}_eval.csv'), index=False)

In [4]:
save_eval_data('logs/Eval', 'noise_level-1')

1.471 1.019
1.471(1.019)
0.299 0.41
0.299(0.410)
656.147 315.165
656.147(315.165)
3.249 34.503
3.249(34.503)
2.631 1.77
2.631(1.770)
0.628 0.411
0.628(0.411)
297.688 237.265
297.688(237.265)
4.015 23.524
4.015(23.524)
1.204 0.701
1.204(0.701)
0.137 0.237
0.137(0.237)
762.27 304.969
762.270(304.969)
0.677 4.305
0.677(4.305)
1.359 0.714
1.359(0.714)
0.272 0.385
0.272(0.385)
882.989 241.758
882.989(241.758)
4.457 25.378
4.457(25.378)
1.202 0.658
1.202(0.658)
0.135 0.213
0.135(0.213)
886.087 248.464
886.087(248.464)
0.536 3.652
0.536(3.652)
1.403 0.567
1.403(0.567)
0.201 0.272
0.201(0.272)
912.892 214.656
912.892(214.656)
0.542 2.44
0.542(2.440)
1.317 0.673
1.317(0.673)
0.257 0.378
0.257(0.378)
935.184 185.346
935.184(185.346)
1.689 6.714
1.689(6.714)
1.43 0.636
1.430(0.636)
0.145 0.212
0.145(0.212)
889.159 242.081
889.159(242.081)
0.559 3.556
0.559(3.556)
1.302 0.695
1.302(0.695)
0.248 0.366
0.248(0.366)
774.239 300.377
774.239(300.377)
3.302 18.235
3.302(18.235)
1.373 0.74
1.373(0.740)
0

In [49]:
save_eval_data('logs/Eval_Push', 'push_level:5')

In [11]:
data = load_data("logs/Eval2/Expert-MLP-noise_type-uniform-noise_level-3-debug.npy")

In [12]:
check_data(data)

done (100, 1011)
action_fluctation (100, 1011)
tracking_error (100, 1011)
cost_of_transport (100, 1011)
base_vel (100, 1011, 3)
contact_force_z (100, 1011, 4)
base_vel_roll (100, 1011)
base_vel_pitch (100, 1011)
base_vel_yaw (100, 1011)
first_done (100,)
Fall (100,)
noise_level 3
noise_type Length:  7


In [14]:
print(np.mean(data['tracking_error']))

0.08962187
